In [1]:
from transformers import RobertaModel
from transformers import RobertaTokenizer
import pandas as pd
import torch
import torch.nn as nn
import numpy as np

from pan20 import fake
from torch.nn import DataParallel
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

In [2]:
### Load dataset

df = fake.load_data()
#truth = {x['author']: x['label'] for x in fake.load_truth()}
#df['label'] = df.author.apply(lambda x: truth[x])

In [3]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

In [19]:
# encoded_data = tokenizer.batch_encode_plus(df['tweet'], pad_to_max_length=True, add_special_tokens=True)

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
model= RobertaModel.from_pretrained('roberta-base')
#model = DataParallel(model)
model = model.to(device)
model.eval()

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inpl

In [ ]:
# with torch.no_grad():

#     input_ids = torch.tensor(encoded_data["input_ids"])
#     outputs = model(input_ids)
#     last_hidden_states = outputs[0]
#     sentencevec = last_hidden_states[:,0,:]

In [22]:
from torch.utils.data import Dataset

class PanDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):

        text = self.data.iloc[idx].values
        #label_tensors = torch.tensor(label)
        
        word_pieces = ["[CLS]"]
        tokens = self.tokenizer.tokenize(text)
        word_pieces += tokens + ["[SEP]"]
        
        ids = self.tokenizer.convert_tokens_to_ids(word_pieces)
        tokens_tensors = torch.tensor(ids)
        
        return tokens_tensors

In [21]:
df_test.iloc[0,:].values

array(['#HASHTAG# How Did We Get Here? A Timeline of the Taylor Swift and Kanye West Drama #URL# #HASHTAG#'],
      dtype=object)

In [23]:
def collate_fn(batch):
    
    tokens_tensors = [data[0] for data in batch]
#     if batch[0][1] is not None:
#         label_ids = torch.stack([label[1] for label in batch])
#     else:
#         label_ids = None

    tokens_tensors = pad_sequence(tokens_tensors, batch_first=True)
        
    masks_tensors = torch.zeros(tokens_tensors.shape, dtype=torch.long)
    masks_tensors = masks_tensors.masked_fill(tokens_tensors != 0, 1)
    
    return tokens_tensors, masks_tensors

In [24]:
df_test = df.drop('author', 1)

In [25]:
dataset = PanDataset(df_test, tokenizer)

In [26]:
testloader = DataLoader(dataset, batch_size=1, shuffle=False, collate_fn=collate_fn)

In [27]:
a = next(iter(testloader))

AttributeError: 'numpy.ndarray' object has no attribute 'strip'

In [11]:
u_vectors = []

In [12]:
with torch.no_grad():
    for data in testloader:
        tokens_tensors, masks_tensors = data[:2]
        tokens_tensors = tokens_tensors.to(device)
        masks_tensors = masks_tensors.to(device)
        outputs = model(input_ids=tokens_tensors,
                        token_type_ids=None,
                        attention_mask=masks_tensors)
        
        u_vectors.append(outputs[0][:,0,:].cpu().squeeze().numpy())

        del tokens_tensors
        del masks_tensors

In [13]:
len(u_vectors)

30000

In [43]:
type(u_vectors)

list

In [14]:
sum_vectors = []

for i in range(0, 30000, 100):
    sum_vectors.append(np.sum(u_vectors[i:i+100], axis=0))

In [13]:
avg_vectors = []

for i in range(0, 30000, 100):
    avg_vectors.append(np.average(u_vectors[i:i+100], axis=0))

In [16]:
df.author[100]

'8vp74g6kssomu1a6akix6y3hqy6552t7'

In [14]:
## get original information

new_df = {
    'author': [],
    'label': []}

for i in range(0, 30000, 100):
    new_df['author'].append(df.author[i])
    new_df['label'].append(df.label[i])


new_df = pd.DataFrame(new_df)

In [18]:
len(new_df)

300

In [15]:
new_df['label'].value_counts()

1    150
0    150
Name: label, dtype: int64

In [18]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

In [47]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
scores = cross_val_score(clf, sum_vectors, new_df['label'], cv=5)
#proba = cross_val_predict(clf, sum_vectors, new_df['label'], cv=5, method='predict_proba')
print("Vectors by sum:", scores)
print("Vectors by sum:", np.mean(scores))

clf = LogisticRegression()
scores = cross_val_score(clf, avg_vectors, new_df['label'], cv=5)
proba = cross_val_predict(clf, avg_vectors, new_df['label'], cv=5, method='predict_proba')
print("Vectors by avg:", scores)
print("Vectors by avg:", np.mean(scores))

/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22.

Vectors by sum: [0.66666667 0.7        0.66666667 0.8        0.63333333]
Vectors by sum: 0.6933333333333332
Vectors by avg: [0.7        0.73333333 0.71666667 0.65       0.61666667]
Vectors by avg: 0.6833333333333333


/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22.

In [22]:
from sklearn.naive_bayes import GaussianNB

clf = GaussianNB()
s_scores = cross_val_score(clf, sum_vectors, new_df['label'], cv=5)
print("Vectors by sum:", s_scores)
print("Vectors by sum:", np.mean(s_scores))

clf = GaussianNB()
a_scores = cross_val_score(clf, avg_vectors, new_df['label'], cv=5)
print("Vectors by avg:", a_scores)
print("Vectors by avg:", np.mean(a_scores))

Vectors by sum: [0.63333333 0.75       0.65       0.58333333 0.55      ]
Vectors by sum: 0.6333333333333334
Vectors by avg: [0.63333333 0.75       0.65       0.58333333 0.55      ]
Vectors by avg: 0.6333333333333334


In [30]:
from sklearn import svm

svc = svm.SVC(C=100000, degree=1000000, kernel='rbf', gamma=0.0009)
s_scores = cross_val_score(svc, sum_vectors, new_df['label'], cv=5)
print("Vectors by sum:", s_scores)
print("Vectors by sum:", np.mean(s_scores))

svc = svm.SVC(C=100000, degree=100, kernel='rbf', gamma=0.0009, probability=True)
a_scores = cross_val_score(svc, avg_vectors, new_df['label'], cv=5)
proba = cross_val_predict(svc, avg_vectors, new_df['label'], cv=5, method='predict_proba')
print("Vectors by avg:", a_scores)
print("Vectors by avg:", np.mean(a_scores))

Vectors by sum: [0.63333333 0.65       0.73333333 0.65       0.61666667]
Vectors by sum: 0.6566666666666666
Vectors by avg: [0.68333333 0.75       0.71666667 0.75       0.66666667]
Vectors by avg: 0.7133333333333333


In [16]:
## input all data for training

from sklearn import svm

svc = svm.SVC(C=100000, degree=100, kernel='rbf', gamma=0.0009, probability=True)
svc.fit(avg_vectors, new_df['label'])

SVC(C=100000, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=100, gamma=0.0009, kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [27]:
np.mean(new_df['label']== svc.predict(avg_vectors))

0.96

In [28]:
import joblib

joblib.dump(svc, 'pan20/fake/roberta-base.model')

['pan20/fake/roberta-base.model']

In [ ]:
## Grid search
history = {'Accuracy':[],'kernel':[], 'degree':[], 'gamma':[]}
scores = []
for i in range(1, 100000):
    for k in['linear', 'rbf', 'sigmoid']:
        for j in [1,2,3]:
            g = i*(1e-5)

            svc = svm.SVC(C=10000, class_weight='balanced', kernel=k, degree=j, probability=True, verbose=0, gamma=g)
            a_scores = cross_val_score(svc, avg_vectors, new_df['label'], cv=5)
            history['Accuracy'].append(np.mean(a_scores))
            history['kernel'].append(k)
            history['degree'].append(j)
            history['gamma'].append(g)

max_index = np.argmax(history['Accuracy'])
print(max(history['Accuracy']))
print(history['kernel'][max_index])
print(history['degree'][max_index])
print(history['gamma'][max_index])

In [82]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=200, max_depth=5)
s_scores = cross_val_score(rf, sum_vectors, new_df['label'], cv=5)
proba = cross_val_predict(rf, sum_vectors, new_df['label'], cv=5, method='predict_proba')
print("Vectors by sum:", s_scores)
print("Vectors by sum:", np.mean(s_scores))

rf = RandomForestClassifier(n_estimators=200, max_depth=5)
a_scores = cross_val_score(rf, avg_vectors, new_df['label'], cv=5)
#proba = cross_val_predict(rf, avg_vectors, new_df['label'], cv=5, method='predict_proba')
print("Vectors by avg:", a_scores)
print("Vectors by avg:", np.mean(a_scores))

Vectors by sum: [0.66666667 0.78333333 0.7        0.68333333 0.68333333]
Vectors by sum: 0.7033333333333334
Vectors by avg: [0.66666667 0.73333333 0.7        0.66666667 0.71666667]
Vectors by avg: 0.6966666666666665


In [83]:
is_spreader = []
for i in range(300):
    is_spreader.append(proba[i][1])

In [84]:
new_df_all = new_df.assign(probability=is_spreader)

In [85]:
pred_s = []
for i in range(300):
    if proba[i][0]>proba[i][1]:
        pred_s.append(0)
    else:
        pred_s.append(1)

In [23]:
np.mean(pred_s == new_df_all['label'])

NameError: name 'pred_s' is not defined

In [87]:
new_df_all.to_csv('roberta_rf_067_s.csv', index=False)